In [42]:
import pandas as pd
import random

In [43]:
player_matches = pd.read_csv("../data/raw_data/20100101_20191129_player_matches.csv")

In [44]:
def get_game_for_row(row):
    if row['home_game']:
        return row['opponent_key'] + "@" + row['team_key']
    else:
        return row['team_key'] + "@" + row['opponent_key']

In [45]:
def get_player_salary(min_fp, max_fp, player_fp):
    MAX_SALARY = 12000
    MIN_SALARY = 3500

    return round(MIN_SALARY + MAX_SALARY * (player_fp - min_fp) / max_fp + (random.random() - 0.5) * 1000, -2)

In [46]:
def generate_salaries(daily_pms):
    max_fp = daily_pms['fp_l5'].max()
    min_fp = daily_pms['fp_l5'].min()
    
    player_fps = list(daily_pms['fp_l5'])
    salaries = []
    
    for player_fp in player_fps:
        salaries.append(get_player_salary(min_fp, max_fp, player_fp))
    
    return salaries

In [47]:
def generate_positions(num):
    pos = []

    for i in range(num):
        rand = random.random()
        if rand < 0.20:
            pos.append("C")
        elif rand < 0.40:
            pos.append("PG")
        elif rand < 0.60:
            pos.append("SF")
        elif rand < 0.80 :
            pos.append("PF")
        else:
            pos.append("SG")
    return pos

In [48]:
def create_fanduel_competition_for_date(player_matches, date):
    daily_pms = player_matches[player_matches.date == date]
    
    if len(daily_pms) == 0:
        return None
    
    fantasy_df = pd.DataFrame()
    fantasy_df['player_id'] = daily_pms['player_id']
    fantasy_df['Position'] = generate_positions(len(daily_pms))
    fantasy_df['Nickname'] = daily_pms['name']
    fantasy_df['Salary'] = generate_salaries(daily_pms)
    fantasy_df['Game'] = daily_pms.apply(get_game_for_row, axis = 1)
    fantasy_df['Team'] = daily_pms['team_key']
    fantasy_df['Opponent'] = daily_pms['opponent_key']
    fantasy_df['FP'] = daily_pms['fp']
    
    fantasy_df.to_csv("../data/competitions_clean/" + "" + date + "_fake.csv", index = False)
    
    return fantasy_df

In [49]:
dates = [
    "2012-03-01",
    "2013-02-07",
    "2013-10-30",
    "2015-10-08",
    "2012-10-08",
    "2013-11-17",
    "2014-11-17",
    "2013-03-05",
    "2014-11-17",
    "2014-04-17",
    "2015-04-17",
    "2015-06-01",
    "2014-02-11",
    "2014-03-12",
    "2013-02-15",
    "2014-03-17",
    "2015-02-16",
    "2015-03-13",
    "2015-02-22",
    "2015-02-24",
    "2015-11-27",
    "2015-02-14",
    "2012-01-14",
    "2014-01-11",
    "2015-05-11",
    "2014-07-11",
    "2015-04-11",
    "2014-08-11",
    "2015-01-11"
]

In [50]:
for date in dates:
    create_fanduel_competition_for_date(player_matches, date)